In [ ]:
class Player:
    def __init__(self):
        self.inventory = {}
        self.alive = True
    
    def __repr__(self):
        return 'Player(alive={}, inventory={})'.format(self.alive, self.inventory)

In [ ]:
from abc import ABC, abstractmethod

class Room:
    def __init__(self,
                 contents=None):
        self.doors = {}
        self.contents = dict(contents) if contents else None

    def process_command(self, command, player):
        """Process room-specific commands.
        """
        return False
    
    @property
    @abstractmethod
    def description(self):
        pass

In [ ]:
class StaticRoom(Room):
    """A room with a static description"""
    def __init__(self, description, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._description = description
        
    @property
    def description(self):
        return self._description

In [ ]:
class LlamaRoom(Room):
    """This is a subclass of Room which contains some llamas.
    """
    def __init__(self, count):
        super(LlamaRoom, self).__init__(
            contents={'llama': count})

    @property
    def description(self):
        return 'This room is filled with {} serene llamas.'.format(
            self.contents['llama'])

    def process_command(self, command, player):
        if command == 'pet llama':
            if self.contents['llama'] < 1:
                print('Unfortunately there are no llamas to pet.')
            else:
                print('The llama looks pleased and then gallops off, its mission in this dimension completed.')
                self.contents['llama'] -= 1
            return True

        return False

In [ ]:
class BearRoom(StaticRoom):
    """A room containing a bear.
    """
    def __init__(self):
        super(BearRoom, self).__init__(
            description='This rooms contains a grumpy looking bear.',
            contents={'bear': 1})

    def process_command(self, command, player):
        if command == 'pet bear':
            print('The bear is not impressed and re-enacts "The Revenant" on you.')
            player.alive = False
            return True
        return False

In [ ]:
class Game:
    def __init__(self, current_room, player):
        self.current_room = current_room
        self.player = player

In [ ]:
from enum import Enum

class Direction(Enum):
    North = 'north'
    South = 'south'
    East = 'east'
    West = 'west'

In [ ]:
from itertools import chain, permutations

DIRECTIONS = {
    from_dir: start_dir
    for from_dir, start_dir
    in chain(permutations((Direction.North, Direction.South)), 
             permutations((Direction.East, Direction.West)))
}

def connect(room_from: Room, room_to: Room, dir_from: Direction):
    dir_to = DIRECTIONS[dir_from]

    if room_from.doors.get(dir_from) is not None:
        raise ValueError(
            'The {} door in {} is already assigned'.format(
                dir_from.value, room_from))
    if room_to.doors.get(dir_to) is not None:
        raise ValueError(
            'The {} door in {} is already assigned'.format(
                dir_from.value, room_from))
        
    room_from.doors[dir_from] = room_to
    room_to.doors[dir_to] = room_from

In [ ]:
def make_game():
    """Construct a game object.
    """
    start_room = StaticRoom('You are in a dark room.')
    llama_room = LlamaRoom(42)
    bear_room = BearRoom()

    connect(start_room, llama_room, Direction.North)
    connect(start_room, bear_room, Direction.East)

    game = Game(start_room, Player())
    return game

In [ ]:
def process_standard_commands(command, game):
    """Process commands which are common to all rooms.

    This includes things like following directions, checking inventory, exiting
    the game, etc.

    Returns true if the command is recognized and processed. Otherwise, returns
    false.

    """
    if command in (d.value for d in game.current_room.doors):
        room = game.current_room.doors[Direction(command)]
        game.current_room = room
    elif command in (d.value for d in Direction):
        print('There is no door to the {}'.format(command))
    elif command == 'description':
        print(game.current_room.description)
    elif command == 'inventory':
        print('Inventory %s' % game.player.inventory)
    elif command == 'quit':
        game.player.alive = False
    else:
        # unrecognized command
        return False

    return True

In [ ]:
def print_room_details(room):
    """Print the details of the current room.
    """
    print(room.description)
    for direction in room.doors:
        print('There is a door to the {}'.format(direction.value))

In [ ]:
def main_loop(game):
    """Process commands from the user until they die.
    """
    while True:
        if not game.player.alive:
            print('You are dead.')
            return

        print('')
        print_room_details(game.current_room)
        command = input('> ')
        handled = process_standard_commands(command, game) \
            or game.current_room.process_command(command, game.player)
        if not handled:
            print('unrecognized command!')

In [ ]:
main_loop(make_game())